In [1]:
import sys
import os
import numpy as np

from codingTools import *


In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
originalFile="../images/03508649.1_512_512_2_0_12_0_0_0d.raw" #int16
#originalFile="../images/03508649.1_128_128_1_0_8_0_0_0.raw"
#originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
ImageRaw = load_image_raw(originalFile, 1, 512, 512, "int16") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
#ImageRaw = load_image_raw(originalFile, 1, 128, 128, "uint8")
#ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics_data = statistics(original_image)
entropy = statistics_data.compute_entropy()
print("order zero entropy of the image: ",entropy)

order zero entropy of the image:  10.431490855522489


In [4]:
print(original_image)

[[[ 25601  26369 -32255 ... -24831 -29183 -31487]
  [ 24321  27137  27393 ... -24319 -27391 -29183]
  [ 25345  27393 -30207 ... -23807 -29183 -25087]
  ...
  [ 26881  30465 -25087 ... -19455 -24319 -25087]
  [ 19201  21761  30209 ... -17919  31745 -27135]
  [ 19457  22529  25089 ... -27135 -29183  28673]]]


In [5]:
######SAVE RAW IMAGE######
# decodedFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0_copy.raw"
# ImageRawDecoded = save_image_raw(decodedFile)
# ImageRawDecoded.save_image_raw(original_image,"uint8")
###########

In [6]:
### Quantize ###
quantize = Quantizer(1, 10)
image_data_quantized = ImageRaw.get_image_structure_empty()
quantize.quantize(original_image, image_data_quantized)
quant_statics = statistics(image_data_quantized).compute_entropy()
print(f"Entropia: {quant_statics}")


Check Quantize: [[[ 2560.  2636. -3225. ... -2483. -2918. -3148.]
  [ 2432.  2713.  2739. ... -2431. -2739. -2918.]
  [ 2534.  2739. -3020. ... -2380. -2918. -2508.]
  ...
  [ 2688.  3046. -2508. ... -1945. -2431. -2508.]
  [ 1920.  2176.  3020. ... -1791.  3174. -2713.]
  [ 1945.  2252.  2508. ... -2713. -2918.  2867.]]]
Entropia: 8.427372356811585


In [7]:
### Dequantize ###
dequantize = Quantizer(1, 10)
image_data_dequantized = ImageRaw.get_image_structure_empty()
dequantize.dequantize(image_data_quantized, image_data_dequantized)

In [8]:
### PAE ###
pae = Metrics(original_image, image_data_dequantized)
print(f"Pae: {pae.calculate_pae()}")
print(pae.PAE_data)


Pae: 9.0
9.0


In [9]:
### MSE ###
mse = Metrics(original_image, image_data_dequantized)
print(f"Mse: {mse.calculate_mse()}")

Mse: 28.720321655273438


In [10]:
### PSNR ###
psnr = Metrics(original_image, image_data_dequantized)
print(f"Psnr: {psnr.calculate_psnr()} dB")

Psnr: 75.72710570410173 dB


In [11]:
### S-transform forward ###
array = [1,2,3,4,8,6,10,8]
transform = Wavelet(array, 1)
array_transform = transform.s_tranform_forward(array)
print(f"Array modificado: {array_transform}")
    

Array modificado: [ 1  3  7  9 -1 -1  2  2]


In [12]:
### S-transform Inverse ###
array_inv = transform.s_transform_inverse(array_transform)
print(f"Array invertido: {array_inv}")


Array invertido: [ 1  2  3  4  8  6 10  8]


In [13]:
### S-transform Forward image ###
transform_image = Wavelet(original_image, 1)
#print(len(original_image[0][0]))
transform_forward = transform_image.handle_transform_forward()
print(transform_forward)


[[ 25857  28545    449 ...  13056   2432   -512]
 [-16319 -10431 -19519 ...  -1536 -31488   2816]
 [-23167 -20479 -10815 ...   5632  -2048  -3584]
 ...
 [  2944   6400   -256 ...   -256 -13568   7936]
 [  4608   3840   2816 ...  17664  -4352  -2048]
 [  2048    512  -3072 ...   3584   6400 -14336]]


In [14]:
### S-transform Reverse image ###
transform_image_reverse = Wavelet(transform_forward, 1)
transform_reverse = transform_image_reverse.handle_transform_inverse()



In [15]:
print(transform_reverse)

[[[ 25601  26369 -32255 ... -24831 -29183 -31487]
  [ 24321  27137  -5375 ... -24319 -27391 -29183]
  [ 25345  27393 -30207 ... -23807 -29183 -25087]
  ...
  [ 26881  30465   7681 ... -19455 -24319 -25087]
  [ 19201  21761  -2559 ... -17919  -1023   5633]
  [ 19457  22529  -7679 ... -27135   3585  -4095]]]


In [16]:
print(original_image)
 

[[[ 25601  26369 -32255 ... -24831 -29183 -31487]
  [ 24321  27137  27393 ... -24319 -27391 -29183]
  [ 25345  27393 -30207 ... -23807 -29183 -25087]
  ...
  [ 26881  30465 -25087 ... -19455 -24319 -25087]
  [ 19201  21761  30209 ... -17919  31745 -27135]
  [ 19457  22529  25089 ... -27135 -29183  28673]]]


### Classe 17/09
Predicció:
X = 1,2,3,4,8,6,10,8
Causal
Pi = {0 si i=0, X(i-1) si i>0}
X'i = Xi - Pi
P = 0,1,2,3,4,8,6,10 (Predicció)
X' = 1,1,1,1,4,-2,4,-2
#
###### TODO
Predict sobre una matriu. -> ha de ser invertible -> img de 512 512
Calcular entropia del resultat X' -> H(x') <= H(x)

In [17]:
pred = Prediction(original_image)
mat_pred = pred.predict_image_LOP()
statistics_data_pred = statistics(mat_pred)
entropy_pred = statistics_data.compute_entropy()
print("Original entropy ",entropy)
print("Pred entropy: ", entropy_pred)

/Users/lucasjimeneznunez/Desktop/UAB/4t/TCI/Projecte/TCIProject/TCIProject/codingTools/Prediction.py:23: RuntimeWarning: overflow encountered in scalar add
  prediction = self.image[i-1, j-1] + self.image[i-1, j] + self.image[i, j-1] - 2*self.image[i-1, j-1]


Original entropy  10.431490855522489
Pred entropy:  20.862981711045112
